# Mapreader Workshop - June 7th-8th 2023

----

MapReader accepts different types of map images as input. We're focusing today on re-using already georeferenced maps that are available as XYZ tile layers from libraries, archives, or other services.

# Download

MapReader’s ``Download`` subpackage is used to download maps stored on as XYZ tile layers on a tile server. It contains two classes for downloading maps:

1. ``SheetDownloader`` - This can be used to download map sheets and relies on information provided in a metadata json file.

2. ``Downloader`` - This is used to download maps using polygons and can be used even if you don’t have a metadata json file.

In this workshop, we will use the ``SheetDownloader`` along with metadata stored in the ``persistent_data`` directory of the mapreader repository.

## Import the ``SheetDownloader`` and create your ``my_ts`` object.

In [1]:
from mapreader import SheetDownloader

In [2]:
metadata_path = "../NLS_metadata/metadata_OS_One_Inch_GB_WFS_light.json"
download_url = "https://geo.nls.uk/maps/os/1inch_2nd_ed/{z}/{x}/{y}.png"

__**EXERCISE**__: Set up your ``my_ts`` by passing the ``metadata_path`` and ``download_url`` arguments to the ``SheetDownloader`` .

In [ ]:
# my_ts = SheetDownloader()
my_ts = SheetDownloader(metadata_path=metadata_path, download_url=download_url)

You can view the boundaries of the map sheets included in your metadata by calling the ``.plot_all_metadata_on_map()`` method:

In [ ]:
my_ts.plot_all_metadata_on_map()

The numbers shown are the WFS ID numbers of the maps.

## Select maps to download

Your ``SheetDownloader`` instance (``my_ts``) can be used to query and download map sheets using a number of methods:

1. Any which are within or intersect/overlap with a polygon.
2.  Any which contain a set of given coordinates.
3. Any which intersect with a line.
4. By WFS ID numbers.
5. By searching for a string within a metadata field.

These methods can be used to either directly download maps, or to create a list of queries which can interacted with and downloaded subsequently.

We will be using the querying option.

### 1. Finding map sheets which overlap or intersect with a polygon.

The ``.query_map_sheets_by_polygon()`` and ``.download_map_sheets_by_polygon()`` methods can be used find and download map sheets which are within or intersect/overlap with a [shapely.Polygon](https://shapely.readthedocs.io/en/stable/reference/shapely.Polygon.html#shapely.Polygon).

These methods have two modes:

- "within" - This finds map sheets whose bounds are completely within the given polygon.
- "intersects" - This finds map sheets which intersect/overlap with the given polygon.

The ``mode`` can be selected by specifying ``mode="within"`` or ``mode="intersects"``.

#### Create a polygon

In [ ]:
from mapreader import create_polygon_from_latlons

__**EXERCISE**__: Create a polygon using the following latlons: ``55.6, -3.5, 56, -2.8``

In [ ]:
# my_polygon = create_polygon_from_latlons()

#### Find maps

To find map sheets which fall **within** the bounds of this polygon:

In [ ]:
my_ts.query_map_sheets_by_polygon(my_polygon, mode="within", print=True)

Nothing was found - our polygon is too small for any maps to fall completely within it.

Instead, to find map sheets which **intersect** with this polygon:

In [ ]:
my_ts.query_map_sheets_by_polygon(my_polygon, mode="intersects", print=True)

To see what you've found, plot your query results on a map using the ``.plot_queries_on_map()`` method:

In [ ]:
my_ts.plot_queries_on_map(map_extent="uk")

### 2. Finding map sheets which contain a set of coordinates.

The ``.query_map_sheets_by_coordinates()`` and ``.download_map_sheets_by_coordinates()`` methods can be used find and download map sheets which contain a set of coordinates.

> **_NOTE:_** We use the ``append=True`` argument to ensure our new queries are appended to our existing list.

In [ ]:
my_ts.query_map_sheets_by_coordinates((-4.5, 55.4), print=True, append=True)

We've used the ``append=True`` argument and so, if you plot your found queries, you will see a new map sheet has been added to your queries list.

In [ ]:
my_ts.plot_queries_on_map(map_extent="uk")

### 3. Finding map sheets which intersect with a line.

The ``.query_map_sheets_by_line()`` and ``.download_map_sheets_by_line()`` methods can be used find and download map sheets which intersect with a [shapely.LineString](https://shapely.readthedocs.io/en/stable/reference/shapely.LineString.html#shapely.LineString).

#### Create a line

In [ ]:
from mapreader import create_line_from_latlons

__**EXERCISE**__: Create a line using the following latlons: ``(56.5, -5), (57.0, -4.5)``

In [ ]:
# my_line = create_line_from_latlons()

#### Find maps

> **_NOTE:_** In the previous examples, we used the ``print=True`` argument to print our query results each time. We've now removed this so query results aren't being printed.

In [ ]:
my_ts.query_map_sheets_by_line(my_line, append=True)

Again, after plotting your queries on a map, you'll see some new map sheets have been added to your queries list.

In [ ]:
my_ts.plot_queries_on_map(map_extent="uk")

### 4. Finding map sheets using their WFS ID numbers.

The ``.query_map_sheets_by_wfs_ids()`` and ``.download_map_sheets_by_wfs_ids()`` methods can be used find and download map sheets using their WFS ID numbers.

These are the numbers that are being plotted on the above maps.

#### One map at a time

In [ ]:
my_ts.query_map_sheets_by_wfs_ids(12, append=True)

#### Multiple maps at a time

In [ ]:
my_ts.query_map_sheets_by_wfs_ids([30, 37, 38], append=True)

In [ ]:
my_ts.plot_queries_on_map(map_extent="uk")

### 5. Finding map sheets by searching for a string in their metadata.

The ``.query_map_sheets_by_string()`` and ``.download_map_sheets_by_string()`` methods can be used find and download map sheets by searching for a string in their metadata.

These methods use [regex string searching](https://docs.python.org/3/library/re.html) to find map sheets whose metadata contains a given string. 
Wildcards and regular expressions can therefore be used in the ``string`` argument.

In [ ]:
my_ts.query_map_sheets_by_string("Stirling", append=True)

In [ ]:
my_ts.plot_queries_on_map(map_extent="uk")

The above query command will search for "Stirling" in **all** metadata fields.

If instead, you'd like to search a particular metadata field (e.g. "IMAGEURL"), you can specify the ``keys`` argument.

> _**NOTE**_: You will need to have an idea of the structure of your metadata in order to do this. Use ``my_ts.features[0]`` to view the metadata for the first map sheet in our metadata, if you would like to see how our metadata is structured.

For the maps we are using in this workshop, it is possible to use the NLS Maps online collection to identify metadata strings to search for, like the words in the title or the unique ids that are present in the image URL. 

For example, if you navigate in a new browser window to https://maps.nls.uk/view/74487492, you can see one of the one-inch sheets. This can be selected by searching for "74487492" (the number at the end of the URL) in the "IMAGEURL" field.

To find other maps, you can use the index of digitized one-inch maps provided by the NLS [here](https://maps.nls.uk/os/one-inch-2nd/).

In [ ]:
my_ts.query_map_sheets_by_string(
    "74487492", keys=["properties", "IMAGEURL"], append=True
)

In [ ]:
my_ts.plot_queries_on_map(map_extent="uk")

### Print found queries

You can print your queries list at any time using the ``.print_found_queries()`` method.

This means you can run multiple queries and check what you've found at the end.

In [ ]:
my_ts.print_found_queries()

## Download query results

To download the image files of the maps from the NLS tile server, you need to first set a zoom level using the ``.get_grid_bb()`` method.

In [ ]:
my_ts.get_grid_bb(14)

Then, you can download your map sheets using ``.download_map_sheets_by_queries()``:

> _**NOTE**_: We have left ``path_save`` and ``metadata_fname`` as the default values, so maps will be saved in ``"./maps/"`` and their metadata will be saved as ``"./maps/metadata.csv"``.

> _**NOTE**_: Downloading maps will take some time depending on the number of sheets being downloaded!

In [ ]:
my_ts.download_map_sheets_by_queries(overwrite=True)

## Quick download

The cells above demonstrate the different ways we can specify which maps to we are interested.

For the workshop we do not want to spend a long time waiting for the maps to download. Therefore we will reset the query list with a shorter list before downloading the maps.

In [ ]:
metadata_path = "../NLS_metadata/metadata_OS_One_Inch_GB_WFS_light.json"
download_url = "https://geo.nls.uk/maps/os/1inch_2nd_ed/{z}/{x}/{y}.png"

# This is a preselected list of map sheets from the coast near the midland valley in Scotland
nls_image_url_ids = [
    "74488562",
    "74488580",
    "74488586",
    "74488689",
    "74488700",
    "74488705",
    "74488722",
    "74488742",
]

my_ts = SheetDownloader(metadata_path=metadata_path, download_url=download_url)

for nls_image_url_id in nls_image_url_ids:
    print(nls_image_url_id)
    my_ts.query_map_sheets_by_string(
        nls_image_url_id, keys=["properties", "IMAGEURL"], append=True
    )

my_ts.get_grid_bb(14)
my_ts.plot_queries_on_map(map_extent="uk")

# Now actually do the download
my_ts.download_map_sheets_by_queries()

-----

# Load

MapReader’s ``Load`` subpackage is used to load, visualize and patchify images (e.g. maps) saved locally.

In this workshop, we will load the images that we have just downloaded from the NLS tile server.
These are saved in "maps/".

![image.png](attachment:image.png)

However, you could use any map images that you have saved locally. 
See the [Input Guidance](https://mapreader.readthedocs.io/en/latest/Input-guidance.html) section of the MapReader documentation for more details about file formats and metadata requirements.

## Import the ``loader`` and create your ``my_files`` object.

In [ ]:
from mapreader import loader

__**EXERCISE**__: Load your maps. They are saved in ``"./maps/*png"``

In [ ]:
# my_files = loader()

### Add metadata

Add the ``metadata.csv`` file that was created when downloading your maps.

__**EXERCISE**__: Add your metadata. It is saved in ``"./maps/metadata.csv"``

In [ ]:
# my_files.add_metadata()

In [ ]:
parent_df, patch_df = my_files.convert_images()
parent_df.head()

## Patchify

The ``.patchify_all()`` method is used to slice your map images into patches. 

The method used to patchify your maps is selected by specifying ``method="pixel"`` or ``method="meters"``. This determines whether your ``patch_size`` is interpreted with units of ``pixel`` or ``meters``. 

#### Patchify by pixel

In [ ]:
my_files.patchify_all(method="pixel", patch_size=200)

#### Patchify by meters

You will only be able to use ``method="meters"`` if you have coordinates saved for each of your map images. 
These coordinates should correspond to the "bounding box" of your map image (minimum x, minimum y, maximum x and maximum y) and thereby associate the left, bottom, right and top edges of your map image to their geospatial locations.

These can be added by running either ``.add_metadata()`` (assuming your metadata contains these coordinates) or, if your images contain georefencing information, ``.add_geo_info()``.

> _**NOTE**_: We have used ``add_to_parents=False`` here so that these patches are not added to the ``my_files`` object. This is simply so that we don't have two sets of patches added after running both ``.patchify_all()`` commands.

In [ ]:
my_files.patchify_all(method="meters", patch_size=5000, add_to_parents=False)

You will see your patches are saved in separate directories, each indicating the patch size and method used.

![image.png](attachment:image.png)

### Visualize results

``MapReader`` also contains some useful functions for visualizing your patches.

For example, the ``.show_sample()`` method can be used to show a random sample of your patches:

In [ ]:
my_files.show_sample(num_samples=3, tree_level="patch", random_seed=1)

You may also want to see all the patches created from one of your parent images. 
This can be done using the ``.show_parent()`` method:

In [ ]:
my_files.show_parent("map_74488562.png")

The ``.calc_pixel_stats()`` method can be used to calculate statistics on the pixel intensities of each patch.

This can be useful when annotating patches, as patches with higher pixel intensities contain more color (i.e. are more likely to contain features as opposed to be blank space).

In [ ]:
my_files.calc_pixel_stats()

By running the ``.convert_images()`` method, you will see that the means and standard deviations of pixel intensities of each patch have been added to your ``my_files`` object.

In [ ]:
parent_df, patch_df = my_files.convert_images()
patch_df.head()

To save these outputs, use the ``save=True`` argument.

> _**NOTE**_: By default, this will save your outputs as ``.csv`` files. If instead, you'd like to save as ``.xslx`` files, add ``save_format="excel"`` to your command.

In [ ]:
parent_df, patch_df = my_files.convert_images(save=True, save_format="excel")

In [ ]:
# my_files.save_patches_to_geojson()

-----

# Annotate

Mapreader's ``Annotate`` subpackage is used to annotate images/patches. 

## Create an annotation tasks file

The first step of annotating your patches is to decide the visual features you would like to annotate and define labels for these.
This is done using a separate file (``annotation_tasks.yaml``) which contains all the information needed to carry out an annotation task.

> _**NOTE**_: This file has already been created and saved in the repo, so you don't need to worry about making it. However, if you do want to create your own, copy and edit the template in the "worked_examples" directory.

Your ``annotation_tasks.yaml`` file should have the following structure:

```yaml 
tasks:
  your_task_name:
    labels: ["your_label_1", "your_label_2", "your_label_3"]

paths:
  your_annotation_set:
        patch_paths: "./path/to/patches/*png"
        parent_paths: "./path/to/parents/*png"
        annot_dir: "./path/to/save/annotations"
```

We will be annotating buildings in our 200 pixel patches. 

Our ``annotation_tasks.yaml`` file will therefore look like:

```yaml
tasks:
  buildings:
    labels: ["no_building", "building"]

paths:
  set_001:
        patch_paths: "./patches_200_pixel/*png"
        parent_paths: "./maps/*png"
        annot_dir: "./annotations"
```

## Annotate your images

Before you begin annotating your images, you must tell MapReader:

- who is doing the annotations (``userID``),
- where to find your ``annotation_tasks.yaml`` (``annotation_tasks_file``),
- which task you’d like to run (``task``),
- which annotation_set you would like to run on (``annotation_set``).

We will also use the ``sort_by="mean"`` option, so that the patches with the highest pixel intensities are shown first.

In [ ]:
from mapreader.annotate.utils import prepare_annotation

In [ ]:
userID = "workshop"  # using "workshop" as your user ID will load previous annotations
annotation_tasks_file = "annotation_tasks.yaml"
task = "buildings"
annotation_set = "set_001"

In [ ]:
annotation = prepare_annotation(
    userID,
    annotation_tasks_file=annotation_tasks_file,
    task=task,
    annotation_set=annotation_set,
)

__**EXERCISE**__: Annotate some patches.

In [ ]:
annotation

## Save your annotations

Once you've finished annotating, you will then need to save your annotations using the ``save_annotation`` function.

![image.png](attachment:image.png)

This will save your annotations as a ``.csv`` file, called ``coastline_#scai#.csv`` which will look something like:

|  | image_id | image_path | label |
|- | -- | -- | -- |
|**0** | patch-1994-1994-2991-2991-#map_74490356.png#.png | /Users/rwood/LwM/MapReader/worked_examples/geo... | 2 |
|**1** | patch-6622-5676-7424-5888-#map_74487492.png#.png | /Users/rwood/LwM/MapReader/worked_examples/geo... | 1 |
|**2** | patch-3960-3960-4950-4950-#map_74489053.png#.png | /Users/rwood/LwM/MapReader/worked_examples/geo... | 2 |
|**3** | patch-3988-0-4985-997-#map_74490356.png#.png | /Users/rwood/LwM/MapReader/worked_examples/geo... | 1 |

In [ ]:
from mapreader.annotate.utils import save_annotation

In [ ]:
save_annotation(
    annotation,
    userID,
    annotation_tasks_file=annotation_tasks_file,
    task=task,
    annotation_set=annotation_set,
)

----

# Classify

Mapreader's ``Classify`` subpackage is used to 1) train or fine-tune a CV (computer vision) model to recognize visual features based on your annotated patches and 2) use your model to predict the labels of patches across entire datasets.

It contains two important classes:

- ``AnnotationsLoader`` - This is used to load and review your annotations and to create datasets and dataloaders which are used to train your model.
- ``ClassifierContainer`` - This is used to set up your model, train/fine-tune it using your datasets and to infer labels on new datasets.

## Load annotations

In [ ]:
from mapreader import AnnotationsLoader

In [ ]:
annotated_images = AnnotationsLoader()

__**EXERCISE**__: Load your annotations. They are saved as ``"./annotations/coastline_#scai#.csv"``

In [ ]:
# annotated_images.load()

### Review labels

Before training your model, you should check your annotations and ensure you are happy with your labels.

This can be done using the ``.review_labels()`` method.

For example, to re-label image with ``id: 5``, type "5" into the text box, press enter, then type the label (number) you'd like to re-label it as and press enter again.

__**EXERCISE**__: Review your annotations.

In [ ]:
annotated_images.review_labels()

### Create datasets and dataloaders

Before using your annotated images to train your model, you will first need to:

1. Split your annotated images into “train”, “val” and and, optionally, “test” datasets.
2. Define some transforms which will be applied to your images to ensure your they are in the right format.
3. Create dataloaders which can be used to load small batches of your dataset during training/inference and apply the transforms to each image in the batch.

> __**NOTE**__: Go to the [Classify](https://mapreader.readthedocs.io/en/latest/User-guide/Classify.html#prepare-datasets-and-dataloaders) section of the user-guide for more information.

The ``.create_dataloaders()`` method carries out these three steps. 

> __**NOTE**__: The default train/val/test split, image transforms and sampler will be used if no arguments are supplied to the ``.create_dataloader()`` method. 

In [ ]:
dataloaders = annotated_images.create_dataloaders()

The code below can be used to see the number of instances of each labelled image in each dataset. 

This shows the importance of having enough annotations so that each dataset contains a good sample of patches for training, validating and testing your model.

In [ ]:
for set_name, dataset in annotated_images.datasets.items():
    print(f'Number of instances of each label in "{set_name}":')
    value_counts = dataset.patch_df["label"].value_counts()
    print(f"no_railspace:\t{value_counts[0]}")
    print(f"railspace:\t{value_counts[1]}\n")

## Train your model

### Set up your ``my_classifier`` object

In [ ]:
from mapreader import ClassifierContainer

In [ ]:
my_classifier = ClassifierContainer("resnet18", labels_map={0:"no_building",1:"building"}, dataloaders)

In [ ]:
my_classifier.add_criterion("cross-entropy")

In [ ]:
params2optimize = my_classifier.generate_layerwise_lrs(min_lr=1e-4, max_lr=1e-3)
my_classifier.initialize_optimizer(params2optimize=params2optimize)

In [ ]:
my_classifier.initialize_scheduler()

### Train your model using your "train" and "val" datasets

In [ ]:
my_classifier.train(num_epochs=10)

### Visualize results

In [ ]:
my_classifier.plot_metric(
    y_axis=["epoch_loss_train", "epoch_loss_val"],
    y_label="Loss",
    legends=["Train", "Valid"],
)

__**EXERCISE**__: Try visualizing another metric. Use ``list(my_classifier.metrics.keys())`` to see your options.

In [ ]:
# my_classifier.plot_metric()

### Test

The "test" dataset can be used to test out your model on previously unseen images. 

As these are already annotated, it makes it easy to understand whether the model is performing as expected.

In [ ]:
my_classifier.inference(set_name="test")

In [ ]:
my_classifier.show_inference_sample_results(label="building", min_conf=0.8)

# Infer 

The fine-tuned model can now be used to infer, or predict, the labels of "unseen" patches.

To show how inference works, we will predict the labels on patches from just one parent image. 

We will do this by creating a ``subset_patch_df`` from our previously saved ``patch_df.xlsx``.
Our new ``subset_patch_df`` will only contain the information of patches from ``map_74488700.png``.

In [ ]:
import pandas as pd

patch_df = pd.read_excel("./patch_df.xlsx", index_col=0)  # load our patch_df.xlsx file
subset_patch_df = patch_df[
    patch_df["parent_id"] == "map_74488700.png"
]  # filter for our chosen parent image
subset_patch_df.head()

> __**NOTE**__: MapReader can be used to predict the labels on entire datasets and so creating a ``subset_patch_df`` is not needed in most use cases.

### Create a dataset (``infer``) from our ``subset_patch_df``

In [ ]:
from mapreader import PatchDataset

In [ ]:
infer = PatchDataset(subset_patch_df, transform="val", patch_paths_col="image_path")

### Load dataset into ``my_classifier``

In [ ]:
my_classifier.load_dataset(infer, "infer")

### Run model inference

In [ ]:
my_classifier.inference("infer")

### Visualize results

In [ ]:
my_classifier.show_inference_sample_results("building", 6, set_name="infer")

### Save results to metadata

First, we need to add the predicted label, its indices and the confidence of the prediction to our ``infer`` dataset.

This is done by adding the values to an ``infer_df`` which we create from the the ``.patch_df`` attribute of our ``infer`` dataset.

In [ ]:
infer_df = infer.patch_df.copy()

In [ ]:
import numpy as np

infer_df["predicted_label"] = my_classifier.pred_label
infer_df["pred"] = my_classifier.pred_label_indices
infer_df["conf"] = np.array(my_classifier.pred_conf).max(axis=1)

infer_df.reset_index(names="name", inplace=True)
infer_df.head()

We can then load our patches and add the metadata from our ``infer.patch_df``.

In [ ]:
from mapreader import load_patches

In [ ]:
my_maps = load_patches(
    "./patches_200_pixel/*74488700*png", parent_paths="./maps/*74488700.png"
)

In [ ]:
my_maps.add_metadata(infer_df, ignore_mismatch=True, tree_level="patch")

In [ ]:
my_maps.add_shape()

We can use the ``.show_parent()`` method to see how our predictions look on our parent map sheet (``map_74488700.png``).

In [ ]:
my_maps.show_parent(
    "map_74488700.png",
    column_to_plot="pred",
    vmin=0,
    vmax=1,
    alpha=0.5,
    patch_border=False,
)

And the ``.convert_images()`` method to save our results.

In [ ]:
my_maps.convert_images(save=True, save_format="csv")

We can also save our outputs as a ``geojson`` file using the ``.save_patches_to_geojson()`` method.
> _**NOTE**_: This will require you to convert your patch coordinates into a polygon format. If these aren't already available, they can be added using the ``.add_patch_polygons()`` method.

In [ ]:
my_maps.add_patch_polygons()
my_maps.save_patches_to_geojson()

Beyond MapReader, these outputs can be used to generate interesting visualizations in other tools.

For example, here are two visualizations of the rail space data from [our paper]:

- https://felt.com/map/MapReader-Launch-Event-map-Urban-Areas-and-Rail-space-9AqftKrvPTlWfwOGkdkCGkD
- https://maps.nls.uk/projects/mapreader/index.html#zoom=6.0&lat=56.00000&lon=-4.00000

# Documentation

Please refer to the [MapReader documentation](https://mapreader.readthedocs.io/en/latest/) for more information.